# Data Pre-processing to make atomic event - similarity map
use cosine similarity score..?

In [7]:
import h5py
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

video_info_path = '/mnt/GitHub/G_TAD_customizing/data/thumos_annotations/'
video_ft_path = '/mnt/GitHub/G_TAD_customizing/data/thumos_feature/TSN_pretrain_avepool_allfrms_hdf5/'
# annotations path
valid_path = os.path.join(video_info_path, 'val_Annotation.csv')
test_path = os.path.join(video_info_path, 'test_Annotation.csv')

# features path
valid_rgb = os.path.join(video_ft_path, 'rgb_val.h5')
valid_flow = os.path.join(video_ft_path, 'flow_val.h5')
test_rgb = os.path.join(video_ft_path, 'rgb_test.h5')
test_flow = os.path.join(video_ft_path, 'flow_test.h5')

In [3]:
anno_df_valid = pd.read_csv(valid_path)

In [4]:
video_name_list = sorted(list(set(anno_df_valid.video.values[:])))
video_name_list

['video_validation_0000051',
 'video_validation_0000052',
 'video_validation_0000053',
 'video_validation_0000054',
 'video_validation_0000055',
 'video_validation_0000056',
 'video_validation_0000057',
 'video_validation_0000058',
 'video_validation_0000059',
 'video_validation_0000060',
 'video_validation_0000151',
 'video_validation_0000152',
 'video_validation_0000153',
 'video_validation_0000154',
 'video_validation_0000155',
 'video_validation_0000156',
 'video_validation_0000157',
 'video_validation_0000158',
 'video_validation_0000159',
 'video_validation_0000160',
 'video_validation_0000161',
 'video_validation_0000162',
 'video_validation_0000163',
 'video_validation_0000164',
 'video_validation_0000165',
 'video_validation_0000166',
 'video_validation_0000167',
 'video_validation_0000168',
 'video_validation_0000169',
 'video_validation_0000170',
 'video_validation_0000171',
 'video_validation_0000172',
 'video_validation_0000173',
 'video_validation_0000174',
 'video_valida

In [5]:
# read in raw feature files extracted by TSN
flow_val_ft = h5py.File(valid_flow, 'r')
rgb_val_ft = h5py.File(valid_rgb, 'r')

In [21]:
# vid_sim = []
similarity_info = {}

for cnt, i in enumerate(video_name_list):
    # get feature for each video
    tmp = np.array(flow_val_ft[i])
    
    # get similarity matrix
    sim_mat = tmp @ (tmp.T)
    
    # in order to divide to norm of each vector
    tmp_norm = np.linalg.norm(tmp, axis=1)
    tmp_norm = tmp_norm.reshape(-1, 1)
    
    # calculate norm mult norm
    norm = tmp_norm @ tmp_norm.T
    
    # calculate final cosine similarity
    similarity = np.divide(sim_mat, norm)
    # vid_sim.append(similarity)
    # print(similarity)
    # plt.imshow(similarity)
    similarity_info[i] = similarity
    if cnt > 1:
        break


In [22]:
similarity_info

{'video_validation_0000051': array([[1.0000001 , 0.9804375 , 0.8845077 , ..., 0.10073119, 0.11812256,
         0.10660324],
        [0.9804375 , 1.0000001 , 0.8825182 , ..., 0.09670395, 0.11142503,
         0.09049875],
        [0.8845077 , 0.8825182 , 1.0000001 , ..., 0.11000324, 0.11699999,
         0.12510358],
        ...,
        [0.10073119, 0.09670395, 0.11000324, ..., 1.0000002 , 0.9118176 ,
         0.56303024],
        [0.11812256, 0.11142503, 0.11699999, ..., 0.9118176 , 1.0000001 ,
         0.5657598 ],
        [0.10660324, 0.09049875, 0.12510358, ..., 0.56303024, 0.5657598 ,
         1.0000001 ]], dtype=float32),
 'video_validation_0000052': array([[1.0000001 , 0.96416575, 0.92781836, ..., 0.21780795, 0.36872688,
         0.35467035],
        [0.96416575, 1.        , 0.97161764, ..., 0.20903008, 0.35197154,
         0.3461988 ],
        [0.92781836, 0.97161764, 1.0000001 , ..., 0.21513091, 0.3280317 ,
         0.3503855 ],
        ...,
        [0.21780795, 0.20903008, 0.21